In [ ]:
# Downloading and installing Convex libraries by Madeleine Udel
# Pkg.update();
try
    using PyPlot;
catch
    Pkg.add("PyPlot");
    Pkg.build("PyPlot");
end
try
    using MAT;
catch
    Pkg.add("MAT");
    Pkg.build("MAT");
end
try
    using SCS;
catch
    Pkg.add("SCS");
end
try
    using Convex;
catch
    Pkg.add("Convex");
    Pkg.build("Convex");
end

<!--announcements-->
<blockquote>
    <center>
    <a href="http://allyouneedismyth.com/wp-content/uploads/2013/10/yin-yang-copy.png"><img src="yinyang.png" width="400px" /></a>
    </center>
      <p><cite><center>"Just as we have two eyes and two feet,<br>
      duality is part of life."<br>
<b>--Carlos Santana</b><br>
      </center></cite></p>
</blockquote>


<h3>Introduction</h3>
In this project, you will implement a linear support vector machine and one operating in kernel space. For this you will need to formulate the primal and dual optimization problems as quadratic programs. You will be using <code>Convex.jl</code>. Before we get started please read through the <a href="http://convexjl.readthedocs.io/en/latest/quick_tutorial.html">tutorial</a> of Convex.jl and its quadratic programming solver.

<h4> Linear classification</h4>

<p> The first assignment is to implement a linear support vector machine. Before we get started we can generat some data to see if everything is working:  
</p>

In [ ]:
function genrandomdata(;N=100,b=0.)
    xTr=randn(N,2); #generate random data and linearly separagle labels
    w0=rand(2,1); # defining random hyperplane
    yTr=sign(xTr*w0+b); # assigning labels +1, -1 labels depending on what side of the plane they lie on
    return(xTr,yTr)
end;

using PyPlot
include("visclassifier.jl");

<p>Remember the SVM primal formulation
$$\begin{aligned}
             &\min_{\mathbf{w},b,\xi} \|\mathbf{w}\|^2_2+C \sum_{i=1}^n \xi_i\\
       & \text{such that }  \ \forall i:\\
             & y_i(\mathbf{w}^\top \mathbf{x}_i+b)\geq 1-\xi_i\\
             & \xi_i\geq 0.\\
\end{aligned}
$$
You will need to implement  the function <code>SVMprimal</code>, which  takes is important training data <code>xTr</code> ($n\times d$) and labels <code>yTr</code> ($n\times 1$) with <code>yTr[i]</code>$\in \{-1,1\}$. Currently the solves a placeholder  problem. You need to update the objective, the constraints and introduce new variables to output the correct hyperplane and bias. </p>

In [ ]:
using Convex;
using SCS;
#<GRADED>
function primalSVM(xTr,yTr;C=1);
    """
    function (classifier,w,b) = primalSVM(xTr,yTr;C=1)
    constructs the SVM primal formulation and uses a built-in 
    convex solver to find the optimal solution. 
    
    Input:
        xTr   | training data (nxd)
        yTr   | training labels (nx1)
        C     | the SVM regularization parameter
    
    Output:
        fun   | usage: predictions=fun(xTe);
        wout  | the weight vector calculated by the solver
        bout  | the bias term calculated by the solver
    """
    y=yTr[:];
    N,d=size(xTr);
    wout=rand(d);
    bout=0;
    # dummy code
    w=Variable(d);
    b=Variable(1);
    objective=sumsquares(w);
    constraints=[w>=0];
    problem = minimize(objective,constraints);
    solve!(problem);    
    wout=w.value
    bout=b.value
    # End of dummy code
    
    # TODO 1
    
    fun=xTe->xTe*wout+bout;
    return(fun,wout,bout)
end;
#</GRADED>

We can test your SVM primal solver with the following randomly generated data set. We label it in a way that it is guaranteed to be linearly separable. If your code works correctly the hyper-plane should separate all the $x$'s into the red half and all the $o$'s into the blue half. With sufficiently large values of $C$ (e.g. $C>10$) you should obtain $0\%$ training error. 

In [ ]:
# This may take a while to run the first time you call it
xTr,yTr=genrandomdata();
fun,w,b=primalSVM(xTr,yTr,C=10); # train svm
visclassifier(fun,xTr,yTr,w=w,b=b); # visualize decision boundary
err=mean(sign(fun(xTr)).!=yTr); # compute error 
@printf("Training error:%2.1f%%\n",err*100); # print it out

<h4>Spiral data set</h4>

<p>The linear classifier works great in simple linear cases. But what if the data is more complicated? We provide you with a "spiral" data set. You can load it and visualize it with the following two code snippets:
<pre>

In [ ]:
# load spiral data
using MAT;

function spiraldata(N=300)
    r=linspace(1,2*pi,N);
    xTr1=[sin(2.*r[:]).*r cos(2.*r[:]).*r];
    xTr2=[sin(2.*r[:]+pi).*r cos(2.*r[:]+pi).*r];
    xTr=vcat(xTr1,xTr2);
    yTr=vcat(ones(N,1),-ones(N,1));
    xTr=xTr+randn(size(xTr)).*0.2;
    
    xTe=xTr[1:2:end,:];
    yTe=yTr[1:2:end];
    xTr=xTr[2:2:end,:];
    yTr=yTr[2:2:end];
    return(xTr,yTr,xTe,yTe)
end;

In [ ]:
using PyPlot
figure();
clf();
xTr,yTr,xTe,yTe=spiraldata();
# visualize data

i1=find(yTr.==1);
i2=find(yTr.==-1);
plot(xTr[i1,1],xTr[i1,2],"b.");
hold(true);
plot(xTr[i2,1],xTr[i2,2],"r.");

legend(["+1","-1"]);


<p>If you apply your previously functioning linear classifier on this data set you will see that you get terrible results. Even your training error will be too high.</p>

In [ ]:
# apply linear classifier on the spiral data set
using PyPlot
include("visclassifier.jl");
fun,w,b=primalSVM(xTr,yTr,C=10);
visclassifier(fun,xTr,yTr,w=[],b=0);
err=mean(sign(fun(xTr)).!=yTr);
@printf("Training error:%2.1f%%\n",err*100);

<h3>Implementing a kernelized SVM</h3>

<p> For something as complex as the spiral data set, you need a more complex classifier. 
First implement the kernel function
<pre>	computeK(kerneltype,X,Z,kpar)</pre>
It takes as input a kernel type (kerneltype) and two data sets $\mathbf{X}$ in $\mathcal{R}^{n\times d}$ and $\mathbf{Z}$ in $\mathcal{R}^{m\times d}$ and outputs a kernel matrix $\mathbf{K}\in{\mathcal{R}^{n\times m}}$. The last input, <code>kpar</code> specifies the kernel parameter (e.g. the inverse kernel width $\gamma$ in the RBF case or the degree $p$ in the polynomial case.)
	<ol>
	<li>For the linear kernel (<code>ktype='linear'</code>) svm, use $k(\mathbf{x},\mathbf{z})=x^Tz$ </li> 
	<li>For the radial basis function kernel (<code>ktype='rbf'</code>) svm use $k(\mathbf{x},\mathbf{z})=\exp(-\gamma ||x-z||^2)$ (gamma is a hyperparameter, passed a the value of kpar)</li>
	<li>For the polynomial kernel (<code>ktype='poly'</code>) use  $k(\mathbf{x},\mathbf{z})=(x^Tz + 1)^p$ (p is the degree of the polymial, passed as the value of kpar)</li>
</ol>

<p>You can use the function <b><code>l2distance</code></b> as a helperfunction.</p>





In [ ]:
include("l2distance.jl"); # we are pulling this function from the previous assignment
#<GRADED>
function computeK(kerneltype, X, Z; kpar=0)
    """
    function K = computeK(kernel_type, X, Z)
    computes a matrix K such that Kij=g(x,z);
    for three different function linear, rbf or polynomial.
    
    Input:
    kerneltype: either 'linear','polynomial','rbf'
    X: n input vectors of dimension d (dxn);
    Z: m input vectors of dimension d (dxn);
    kpar: kernel parameter (inverse kernel width gamma in case of RBF, degree in case of polynomial)
    
    OUTPUT:
    K : nxm kernel matrix
    """
    @assert(kerneltype in ["linear","polynomial","poly","rbf"],"Kernel type not known."); # only three kernels defined
    @assert(size(X,2)==size(Z,2),"Input dimensions do not match");# make sure the dimensions match up

    # TODO 2
    
    return(K); 
end;
#</GRADED>

<p>The following code snippet plots an image of the kernel matrix for the data points in the spiral set. Use it to test your <b><code>computeK</code></b> function:</p>

In [ ]:
xTr,yTr,xTe,yTe=spiraldata();
K=computeK("rbf",xTr,xTr,kpar=0.05);
pcolormesh(K); 

Remember that the SVM optimization has the following dual formulation:
$$
\begin{aligned}
             &\min_{\alpha_1,\cdots,\alpha_n}\frac{1}{2} \sum_{i,j}\alpha_i \alpha_j y_i y_j \mathbf{K}_{ij} - \sum_{i=1}^{n}\alpha_i  \\
       \text{s.t.}  &\quad 0 \leq \alpha_i \leq C\\
             &\quad \sum_{i=1}^{n} \alpha_i y_i = 0.
\end{aligned}
$$
This is equivalent to solving for the SVM primal
$$ L(\mathbf{w},b) = C\sum_{i=1}^n \max(1-y_i(\mathbf{w}^\top\phi(\mathbf{x}_i)+b),0) + ||w||_2^2$$
where $\mathbf{w}=\sum_{i=1}^n y_i \alpha_i \phi(\mathbf{x}_i)$ and $\mathbf{K}_{ij}=k(\mathbf{x}_i,\mathbf{x}_j)=\phi(\mathbf{x}_i)^\top\phi(\mathbf{x}_j)$, for some mapping $\phi(\cdot)$.  Please note that here all $\alpha_i\geq 0$, which is possible because we multiply by $y_i$ in the definition of $\mathbf{w}$. One advantage of keeping all $\alpha_i$ non-negative is that we can easily identify non-support vectors as vectors with $\alpha_i=0$. 

<p>Implement the function <code>dualqp</code>, which takes as input a kernel matrix $K$, a vector of labels $yTr$ in $\mathcal{R}^{n}$, and a regularization constant $C\geq 0$. This function should solve the quadratic optimization problem and output the optimal vector $\mathbf{\alpha}\in{\mathcal{R}^n}$.</p>


In [ ]:
using Convex;
using SCS;
#<GRADED>
function dualqp(K,yTr,C);
    """
    function alpha = dualqp(K,yTr,C)
    constructs the SVM dual formulation and uses a built-in 
    convex solver to find the optimal solution. 
    
    Input:
        K     | the (nxn) kernel matrix
        yTr   | training labels (nx1)
        C     | the SVM regularization parameter
    
    Output:
        α     | the calculated solution vector (nx1)
    """
    y=yTr[:];
    N=size(K,1);
    α=Variable(N);

    # TODO 3
    
    return(α.value[:])
end;
#</GRADED>

The following code shows a usecase of how <code>dualqp</code> could be used in practice. 

In [ ]:
C=10;
λ=0.25;
ktype="rbf";
# compute kernel (make sure it is PSD)
K=computeK(ktype,xTr,xTr,kpar=0.25);
ϵ=1e-10;
# make sure it is symmetric and positive semi-definite;
K=(K+K')./2+ϵ*eye(size(K,1)); 

α=dualqp(K,yTr,C);

<p> Now that you can solve the dual correctly, you should have the values for $\alpha_i$. But you are not done yet. You still need to be able to classify new test points. Remember from class that $h(\mathbf{x})=\sum_{i=1}^n \alpha_i y_i k(\mathbf{x}_i,\mathbf{x})+b$. You need to obtain $b$. It is easy to show (and omitted here) that if $C>\alpha_i>0$ (with strict $>$), then we must have that $y_i(\mathbf{w}^\top \phi(\mathbf{x}_i)+b)=1$. Rephrase this equality in terms of $\alpha_i$ and solve for $b$. Implement

<p> b=recoverBias(K,yTr,alphas,C); </p>

<p> where <code>b</code> is the hyperplane bias.
(Hint: This is most stable if you pick an $\alpha_i$ that is furthest from $C$ and $0$. )</p>

In [ ]:
#<GRADED>
function recoverBias(K,yTr,α,C);
    """
    function bias=recoverBias(K,yTr,α,C);
    Solves for the hyperplane bias term, which is uniquely specified by the 
    support vectors with alpha values 0<alpha<C
    
    INPUT:
    K : nxn kernel matrix
    yTr : 1xn input labels
    α  : nx1 vector of alpha values
    C : regularization constant
    
    Output:
    bias : the scalar hyperplane bias of the kernel SVM specified by alphas
    """

    # TODO 4
    
    return(bias);
end;
#</GRADED>

<p> Test your <b><code>recoverBias</code></b> function with the following code, which uses the dual solver on a linearly separable dataset:</p>

In [ ]:
xTr,yTr=genrandomdata(b=0.5);
C=10;
fun,w,b=primalSVM(xTr,yTr,C=10);
K=computeK("linear",xTr,xTr);
ϵ=1e-10;
K=(K+K')./2+ϵ*eye(size(K,1)); # make sure it is symmetric and positive semi-definite;
α=dualqp(K,yTr,C);
ba=recoverBias(K,yTr,α,C);
wa=(α.*yTr[:])'*xTr;
fun=xTe->xTe*wa[:]+ba[1];
visclassifier(fun,xTr,yTr,w=wa,b=ba);
ba/b

<p>
    Implement the function 
    <pre>
    svmclassify=dualSVM(xTr,yTr,C,ktype,kpar);
    </pre>
    It should use your functions <code><b>computeK</b></code> and <code><b>generateQP</b></code> to solve the SVM dual problem of an SVM specified by a training data set (<code><b>xTr,yTr</b></code>), a regularization parameter (<code>C</code>), a kernel type (<code>ktype</code>) and kernel parameter (<code>lmbda</code>, to be used as kpar in Kernel construction). Then, find the support vectors and recover the bias to return <b><code>svmclassify</code></b>, a function that uses your SVM to classify a set of test points <code>xTe</code>.

    
</p>


In [ ]:
#<GRADED>
function dualSVM(xTr,yTr,C,ktype,λ);
    """
    function classifier = dualSVM(xTr,yTr,C,ktype,λ);
    Constructs the SVM dual formulation and uses a built-in 
    convex solver to find the optimal solution. 
    
    Input:
        xTr   | training data (nxd)
        yTr   | training labels (nx1)
        C     | the SVM regularization parameter
        ktype | the type of kernelization: 'rbf','polynomial','linear'
        λ     | the kernel parameter - degree for poly, inverse width for rbf
    
    Output:
        svmclassify | usage: predictions=svmclassify(xTe);
    """
    svmclassify = x->x; #Dummy code
    
    # TODO 5
    
    return(svmclassify);
end;
#</GRADED>

<p>Now we try the SVM with RBF kernel on the spiral data. If you implemented it correctly, train and test error should be close to zero.</p>

In [ ]:
xTr,yTr,xTe,yTe=spiraldata();
C=10.0;
σ=0.25;
ktype="rbf";
svmclassify=dualSVM(xTr,yTr,C,ktype,σ);

visclassifier(svmclassify,xTr,yTr);

# compute training and testing error
predsTr=svmclassify(xTr);
trainingerr=mean(sign(predsTr).!=yTr);
@printf("\nTraining error: %2.4f\n",trainingerr)

predsTe=svmclassify(xTe);
trainingerr=mean(sign(predsTe).!=yTe);
@printf("\Testing error: %2.4f\n",trainingerr)


SVMs are pretty sensitive to hyper-parameters. We can visualize the results of a hyper-parameter grid search as a heat-map, where we sweep across different values of C and kpar and output the result on a validation dataset. Now we ask you to implement a cross-validation function.

In [ ]:
#<GRADED>
function cross_validation(xTr,yTr,xValid,yValid,ktype,CList,lmbdaList)
    """
    function bestC,bestLmbda,ErrorMatrix = cross_validation(xTr,yTr,xValid,yValid,ktype,CList,lmbdaList);
    Use the parameter search to find the optimal parameter,
    Individual models are trained on (xTr,yTr) while validated on (xValid,yValid)
    
    Input:
        xTr      | training data (nxd)
        yTr      | training labels (nx1)
        xValid   | training data (mxd)
        yValid   | training labels (mx1)
        ktype    | the type of kernelization: 'rbf','polynomial','linear'
        CList    | The list of values to try for the SVM regularization parameter C (ax1)
        lmbdaList| The list of values to try for the kernel parameter lmbda- degree for poly, inverse width for rbf (bx1)
    
    Output:
        bestC      | the best C parameter
        bestLmbda  | the best Lmbda parameter
        ErrorMatrix| the test error rate for each given C and Lmbda when trained on (xTr,yTr) and tested on (xValid,yValid),(axb)
    """
    # gridsearch for best parameters
    ErrorMatrix=zeros(length(CList),length(lmbdaList));
    bestC,bestLmbda = 0.,0.;
    
    # TODO 6
            
    return(bestC,bestLmbda,ErrorMatrix)
end;
#</GRADED>

In [ ]:
# gridsearch for best parameters
xTr,yTr,xValid,yValid=spiraldata(100);
CList=(2.0.^linspace(-1,5,7));
lmbdaList=(linspace(0.1,0.5,5));

bestC,bestLmbda,ErrorMatrix = cross_validation(xTr,yTr,xValid,yValid,"rbf",CList,lmbdaList);

pcolormesh(ErrorMatrix);
colorbar()
xlabel("σ")
ylabel("C")
title("Hyperparameter error")

If you implemented everything correctly, the result should look similar to this image:
<center>
 <img src="crossval.png" width="300px" />
</center>

Competition: we ask you to implement function autosvm, which given xTr and yTr, splits them into training data and validation data, and then uses a hyperparameter search to find the optimal hyper parameters. 

Function autosvm should return a function which will act as a classifier on xTe.

You have a 5 minute time limit on multiple datasets, each dataset having different optimal hyperparameters, so you should strive for a good method of finding hyperparameters (within the time limit) instead of just trying to find a static set of good hyperparameters. 

You will get extra credit for the competition if you can beat the base benchmark of 34% error rate.

In [ ]:
#<GRADED>
function autosvm(xTr,yTr)
"""
svmclassify = autosvm(xTr,yTr), where yTe = svmclassify(xTe)
"""
    return x->x[:,1]; # Dummy code
    
# (Optional) TODO 7

end;
#</GRADED>

<!-- -->